# Hello GPU Acceleration

In [1]:
import time
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# Check for any devices recognized as GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

TensorFlow version: 2.18.0
Num GPUs Available:  0
Available devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Try to enable GPU

In [2]:
# Use only the first GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found")

No GPUs found


## Measure CPU/GPU Speed Training an ANN

In [3]:

# Define simple neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(100,)),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Generate some random data
data = tf.random.normal((1000, 100))
labels = tf.random.uniform((1000,), minval=0, maxval=10, dtype=tf.int32)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Time the training process
start_time = time.time()
model.fit(data, labels, epochs=50, batch_size=32)
end_time = time.time()

# Print the training time
print(f"Training time: {end_time - start_time} seconds")

# Check if GPU was used
if tf.config.list_physical_devices('GPU'):
    print("GPU was being used for training.")
else:
    print("CPU was being used for training.")

Epoch 1/50


d:\MyGithub\ML_TensorFlow_Keras_Experiments\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0985 - loss: 2.5752   
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1831 - loss: 2.2482 
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2478 - loss: 2.0925 
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3304 - loss: 1.9569 
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4256 - loss: 1.8271 
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4731 - loss: 1.7235 
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5251 - loss: 1.6188 
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5615 - loss: 1.5612 
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6089 - loss: 1.4557 
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6325 - loss: 1.3953 
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6969 - loss: 1.2999 
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7231 - 

Training times:
- BrainMaxX with Nvidia Titan V: 4.16 seconds
- MacMini M4 with CPU: 1.63 seconds
- DevManPro with CPU: 3.75 seconds

## Measure CPU/GPU Speed Training a CNN

In [2]:
# Generate synthetic training data
input_shape = (64, 64, 3)
num_classes = 10
num_samples = 10000

X_train = tf.random.normal((num_samples,) + input_shape)
y_train = tf.random.uniform((num_samples,), minval=0, maxval=num_classes, dtype=tf.int32)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)

# Simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
history = model.fit(X_train, y_train, epochs=15, batch_size=64, verbose=2)
end_time = time.time()

# Print the training time
print(f"Training time: {end_time - start_time} seconds")

# Check if GPU was used
if tf.config.list_physical_devices('GPU'):
    print("GPU was being used for training.")
else:
    print("CPU was being used for training.")


Epoch 1/15


d:\MyGithub\ML_TensorFlow_Keras_Experiments\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


157/157 - 6s - 35ms/step - accuracy: 0.0968 - loss: 2.3813
Epoch 2/15
157/157 - 5s - 29ms/step - accuracy: 0.1010 - loss: 2.3029
Epoch 3/15
157/157 - 5s - 30ms/step - accuracy: 0.1006 - loss: 2.3026
Epoch 4/15
157/157 - 5s - 29ms/step - accuracy: 0.0996 - loss: 2.3028
Epoch 5/15
157/157 - 5s - 30ms/step - accuracy: 0.0994 - loss: 2.3027
Epoch 6/15
157/157 - 5s - 29ms/step - accuracy: 0.0991 - loss: 2.3029
Epoch 7/15
157/157 - 5s - 29ms/step - accuracy: 0.0997 - loss: 2.3026
Epoch 8/15
157/157 - 5s - 29ms/step - accuracy: 0.1037 - loss: 2.3025
Epoch 9/15
157/157 - 5s - 30ms/step - accuracy: 0.1092 - loss: 2.3017
Epoch 10/15
157/157 - 5s - 29ms/step - accuracy: 0.0972 - loss: 2.3030
Epoch 11/15
157/157 - 5s - 29ms/step - accuracy: 0.1002 - loss: 2.3027
Epoch 12/15
157/157 - 5s - 29ms/step - accuracy: 0.1043 - loss: 2.3025
Epoch 13/15
157/157 - 5s - 29ms/step - accuracy: 0.1057 - loss: 2.3025
Epoch 14/15
157/157 - 5s - 29ms/step - accuracy: 0.0999 - loss: 2.3029
Epoch 15/15
157/157 - 5s -

Training times:
- BrainMaxX with Nvidia Titan V: __ seconds
- MacMini M4 with CPU: __ seconds
- DevManPro with CPU: 70.15 seconds